## 1. 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  

%pip install lightgbm shap matplotlib
%pip install statsmodels
%pip install scikit-learn
%pip install display
%pip install xmltodict  
     

## 2. 한국의 미세먼지 데이터 로드 및 전처리

In [2]:
import pandas as pd
import os

# 공통 파일명 패턴 앞뒤
base_path = "./data"
file_name_format = "{}년 대기오염물질 배출량 통계(시군구별 배출원소분류별 연료별).xlsx"

# 시도명 축약 매핑
name_map = {
    "서울특별시": "서울", "부산광역시": "부산", "대구광역시": "대구", "인천광역시": "인천",
    "광주광역시": "광주", "대전광역시": "대전", "울산광역시": "울산", "세종특별자치시": "세종",
    "경기도": "경기", "강원도": "강원", "충청북도": "충북", "충청남도": "충남",
    "전라북도": "전북", "전라남도": "전남", "경상북도": "경북", "경상남도": "경남",
    "제주특별자치도": "제주" , "제주도": "제주" , "바다":"바다"
}

release_pm25 = {}
release_pm10 = {}

for year in range(2015, 2023):
    file_path = os.path.join(base_path, file_name_format.format(year))
    
    df_raw = pd.read_excel(file_path)

    # 상위 3행 제거 및 헤더 생성
    df = df_raw.iloc[3:].reset_index(drop=True)
    columns_1 = df_raw.iloc[1].tolist()
    columns_2 = df_raw.iloc[2].tolist()
    final_columns = [a if pd.notna(a) else b for a, b in zip(columns_1, columns_2)]
    df.columns = final_columns

    # 필요한 열 추출
    df_filtered = df[["시도", "배출원대분류", "PM-2.5", "PM-10"]].copy()
    df_filtered["PM-2.5"] = pd.to_numeric(df_filtered["PM-2.5"], errors="coerce")
    df_filtered["PM-10"] = pd.to_numeric(df_filtered["PM-10"], errors="coerce")
    df_filtered = df_filtered.dropna(subset=["시도", "배출원대분류", "PM-2.5", "PM-10"])


    # 그룹화 및 피벗 테이블 생성
    summary_pm25 = df_filtered.groupby(["시도", "배출원대분류"])["PM-2.5"].sum().reset_index()
    summary_pm10 = df_filtered.groupby(["시도", "배출원대분류"])["PM-10"].sum().reset_index()

    pivot_pm25 = summary_pm25.pivot(index="시도", columns="배출원대분류", values="PM-2.5").fillna(0).astype(int)
    pivot_pm10 = summary_pm10.pivot(index="시도", columns="배출원대분류", values="PM-10").fillna(0).astype(int)

    # 총합 및 시도 정리
    pivot_pm25["총합"] = pivot_pm25.sum(axis=1)
    pivot_pm10["총합"] = pivot_pm10.sum(axis=1)
    pivot_pm25 = pivot_pm25.reset_index()
    pivot_pm10 = pivot_pm10.reset_index()
    pivot_pm25["시도"] = pivot_pm25["시도"].map(name_map)
    pivot_pm10["시도"] = pivot_pm10["시도"].map(name_map)
    for col in ["농업", "에너지수송 및 저장", "유기용제 사용"]:
        if col in pivot_pm25.columns:
            pivot_pm25 = pivot_pm25.drop(columns=[col])
        if col in pivot_pm10.columns:
            pivot_pm10 = pivot_pm10.drop(columns=[col])

    pivot_pm25.to_csv(f'processed_data/res_{year}_pm2.5.csv', index=False)
    pivot_pm10.to_csv(f'processed_data/res_{year}_pm10.csv', index=False)


In [3]:
# processed_data 폴더의 연도별 배출량 csv 파일 목록
proc_dir = 'processed_data'
proc_files = [f for f in os.listdir(proc_dir) if f.endswith('.csv') and f.startswith('res_')]

# 연도별 배출량 데이터 병합
release_list = []
for f in proc_files:
    year = int(f.split('_')[1])
    pm_type = 'pm2.5' if 'pm2.5' in f else 'pm10'
    df = pd.read_csv(os.path.join(proc_dir, f))
    df['year'] = year
    df['pm_type'] = pm_type
    release_list.append(df)
release_df = pd.concat(release_list, ignore_index=True)


# 8년치 PM 농도 데이터 (wide -> long)
pm25 = pd.read_csv('data/8y_pm2.5.csv')
pm10 = pd.read_csv('data/8y_pm10.csv')
pm25 = pm25.rename(columns={'시/도': '시도'})
pm10 = pm10.rename(columns={'시/도': '시도'})
pm25_long = pm25.melt(id_vars=['시도'], var_name='year', value_name='농도')
pm10_long = pm10.melt(id_vars=['시도'], var_name='year', value_name='농도')
pm25_long['pm_type'] = 'pm2.5'
pm10_long['pm_type'] = 'pm10'
pm_long = pd.concat([pm25_long, pm10_long], ignore_index=True)
pm_long['year'] = pm_long['year'].astype(int) + 2000

# release_df와 pm_long을 year, 시도, pm_type 기준으로 merge
df_master = pd.merge(release_df, pm_long, on=['시도', 'year', 'pm_type'], how='inner')
# 결과 저장
df_master.to_csv('df_master.csv', index=False) 

## 3.상관관계 및 다중 공산성 분석

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv('df_master.csv')

# 오염원 컬럼만 추출 (시도, year, pm_type, PM2.5/PM10 등 제외)
exclude_cols = ['시도', 'year', 'pm_type', 'PM2.5','총합','농도','PM10']
feature_cols = [col for col in df.columns if col not in exclude_cols]

# 상관계수 행렬 계산
corr_df = df[feature_cols].corr(method='pearson')
corr_df.to_csv('corr_df.csv') 
corr_df = pd.read_csv('corr_df.csv', index_col=0)
plt.figure(figsize=(12, 10))
sns.heatmap(corr_df, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Pearson Correlation Heatmap')
plt.tight_layout()
plt.savefig('corr_heatmap.png')
plt.close() 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

# 데이터 로드
df = pd.read_csv('df_master.csv')
# 오염원 컬럼만 추출 (시도, year, pm_type, PM2.5, PM10 등 제외)
exclude_cols = ['시도', 'year', 'pm_type', 'PM2.5','총합','농도','PM10']
feature_cols = [col for col in df.columns if col not in exclude_cols]
X = df[feature_cols]
# VIF 계산
data = []
for i, col in enumerate(X.columns):
    vif = variance_inflation_factor(X.values, i)
    data.append({'feature': col, 'VIF': vif})
vif_df = pd.DataFrame(data)
vif_df.to_csv('vif_table.csv', index=False) 

vif_df

> **같은 배출량이더라도, 오염원의 배출 위치, 방식, 화학 성분, 기상 조건에 따라 대기질 영향은 달라진다**는 것은  
> 환경과학에서 인정된 객관적 사실입니다.

따라서, 단순히 `t/yr` 배출량만으로 대기질 기여도를 판단하기보다는,  
**질량 기준 + 배출 특성 + 기상 조건**을 함께 고려해야 합니다.

| 오염원   | PM2.5 배출량 | 배출 조건             | 대기질 영향              |
|----------|---------------|------------------------|---------------------------|
| 발전소   | 100kg         | 고고도, 확산 양호       | 낮음                      |
| 차량     | 100kg         | 지표면, 교통 밀집        | 매우 높음                 |
| 가정 난방 | 100kg         | 겨울철 밤, 지면 근처     | 높음                      |
| 농업 연소 | 100kg         | 야외, 바람 강함          | 지역에 따라 낮을 수도 있음 |


Ridge: 변수 간 상관·과적합을 완화하면서 모든 오염원별 영향 계수를 얻기 위한 선택.

RidgeCV + cross_val_predict:

RidgeCV → 최적 α 선택.

cross_val_predict → 훈련에 쓰이지 않은 예측으로 공정한 RMSE·R² 산출.

이렇게 하면 “계수”와 “신뢰할 성능 지표”를 동시에 확보해,

계수: 정책용 가중치 해석

RMSE‧R²: 모델이 실측을 얼마나 잘 설명하는지 품질 체크
가 가능합니다.

① PM10 모델에서 보이는 것
배출원	계수	해석 (정성)
도로이동오염원	+4.85	가장 강력한 양(+) 영향원 ⇒ 도로교통 배출 1 σ↑ → PM10 약 +4.9 µg/m³
비도로이동오염원	–2.13	도로·연소류와 공선성 → 부호가 음으로 뒤집힘
비산먼지	–0.98	간접 효과·계수 수축으로 음
비산업 연소	+1.46	두 번째로 큰 양(+) 기여
기타 연소·공정	±1 이하	영향 약함 or 다른 변수에 흡수

α = 6.43 → 가중치가 꽤 ‘완만하게’(over-fitting 방지용으로) 축소되었다는 뜻.

② PM2.5 모델에서 보이는 것
배출원	계수	특징
도로이동오염원	+2.95	여전히 최상위 영향
제조업 연소	+0.70	PM10보다 상대적으로 ↑
비산먼지·기타	음(–)	위와 동일한 해석

α = 5.46 → PM10보다 약간 작은 규제 → 계수가 조금 더 자유롭게 변함.

지표	PM10	PM2.5	해석
RMSE (µg/㎥)	5.72	3.92	

평균 농도 대비 약 15 %(=5.7/39)·18 %(=3.9/21.8) 오차 → 경향 파악·우선순위엔 쓸 만하지만, 정밀 예측용으론 부족

R²	0.25	0.15	분산의 25 %·15 % 설명 → “중간 이하” 수준의 설명력
α = 1000	두 모델 모두 매우 강한 규제 → 계수 축소가 크고, 예측 성능도 **제한적**

3. 날씨 요인 데이터 로드 및 전처리(시계열 데이터 , 풍속, 상대습도, 평균기온)

In [ ]:
import requests, pandas as pd, xmltodict, json, time

API_KEY = "iZg1DoAwxAk6RWJBOuGLXve+S0ah2zN9LwseR4wHfOJVxQgqfyTVPeIBs39M6h7bJ608kO8ZsiC2iEbMaOJX0Q=="
DATE_FMT = "%Y%m%d"

# ────────────────────────── 시·도 → 지점 번호 ──────────────────────────
station_map = {
    "서울": 108,  "부산": 159, "대구": 143, "인천": 112,
    "광주": 156,  "대전": 133, "울산": 152,
    "경기": 119,  "강원": 101, "충북": 131, "충남": 236,
    "전북": 146,  "전남": 165, "경북": 137, "경남": 155,
    "제주": 184,
}

# ────────────────────────── API 호출 함수 ──────────────────────────
def fetch_asos_daily(city: str, stn: int,
                    start_year: int = 2015, end_year: int = 2022) -> pd.DataFrame:
    base = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    rows = []

    for year in range(start_year, end_year + 1):
        params = {
            "ServiceKey": API_KEY,        # Decoding 키 그대로
            "dataType":   "JSON",         # JSON 우선
            "dataCd":     "ASOS",
            "dateCd":     "DAY",
            "startDt":    f"{year}0101",
            "endDt":      f"{year}1231",
            "stnIds":     stn,
            "pageNo":     1,
            "numOfRows":  365,
        }

        r = requests.get(base, params=params, timeout=15)
        r.raise_for_status()

        # JSON ↔ XML 자동 판별
        if "application/json" in r.headers.get("Content-Type", ""):
            data = r.json()
        else:
            data = json.loads(json.dumps(xmltodict.parse(r.text)))

        items = data["response"]["body"]["items"]["item"]
        rows.extend(items)
        time.sleep(0.15)   # 쿼터 여유

    df = pd.json_normalize(rows).rename(columns={
        "tm":     "date",
        "n99Rn":  "rain_mm",
        "avgRhm": "rh",
        "avgTa":  "ta",
        "avgWs":  "ws",
    })

    df["city"] = city
    df["date"] = pd.to_datetime(df["date"])
    
    # rain_mm: 결측값을 0.0으로 대체
    df["rain_mm"] = pd.to_numeric(df["rain_mm"], errors="coerce").fillna(0.0)

    # 나머지 수치 컬럼은 그대로 numeric 변환 (결측치 유지)
    df[["rh", "ta", "ws"]] = df[["rh", "ta", "ws"]].apply(
        pd.to_numeric, errors="coerce"
    )


    return df[["city", "date", "rain_mm", "rh", "ta", "ws"]]

# ────────────────────────── 전체 시·도 루프 ──────────────────────────
frames = []
for city, stn in station_map.items():
    print(f"↳ {city}({stn}) 다운로드 중…")
    frames.append(fetch_asos_daily(city, stn))

kr_meteo_2015_2022 = (
    pd.concat(frames)
    .sort_values(["city", "date"])
    .reset_index(drop=True)
)

print(kr_meteo_2015_2022.head())
print("총 행:", len(kr_meteo_2015_2022))

kr_meteo_2015_2022.to_csv("KOR_met_2015_2022.csv", index=False, encoding="utf-8-sig")


3. 중국/한국의 미세먼지 데이터 로드 및 전처리

In [8]:
import os
import pandas as pd

# 1. 데이터 폴더 내 모든 air-quality.csv 파일 탐색
data_dir = 'data'
files = [f for f in os.listdir(data_dir) if f.endswith('air-quality.csv')]

records = []

for file in files:
    city = file.split('-')[0]  # 파일명에서 도시명 추출
    df = pd.read_csv(os.path.join(data_dir, file))
    df.columns = df.columns.str.strip()
    df['date'] = pd.to_datetime(df['date'])
    for _, row in df.iterrows():
        records.append({
            'city': city,
            'date': row['date'],
            'value': {'pm25': row['pm25'], 'pm10': row['pm10']}
        })

# 2. records로부터 DataFrame 생성
weather_df = pd.DataFrame(records)

# 3. date가 2015~2022년까지만 필터링
weather_df = weather_df[
    (weather_df['date'].dt.year >= 2015) & (weather_df['date'].dt.year <= 2022)
]

# 4. 중국 주요 도시 리스트
china_cities = ['상하이', '청도', '텐진', '베이징']

# 5. 한국 도시만 필터링 (중국 도시 제외)
weather_df_kr = weather_df[~weather_df['city'].isin(china_cities)]

# 6. pivot: index=date, columns=city, values=value(object) - 한국
pivot_df_kr = weather_df_kr.pivot(index='date', columns='city', values='value')
pivot_df_kr.to_csv("PM_KR_2015_2022.csv", encoding="utf-8-sig")

# 7. 중국 도시만 따로 저장
weather_df_cn = weather_df[weather_df['city'].isin(china_cities)]
pivot_df_cn = weather_df_cn.pivot(index='date', columns='city', values='value')
pivot_df_cn.to_csv("PM_CN_2015_2022.csv", encoding="utf-8-sig")

##  준비

In [9]:
import ast, warnings, json, os, pickle
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import shap
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 한글 폰트 설정
from matplotlib import font_manager as fm 
for font in ["Malgun Gothic", "NanumGothic", "AppleGothic"]:
    if any(font in f.name for f in fm.fontManager.ttflist):
        plt.rcParams["font.family"] = font
        break

# 경고 메시지 무시
warnings.filterwarnings("ignore")

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# 경로 설정
DATA = Path("./")
OUT_PM25 = Path("./domestic_pm25_improved")
OUT_PM10 = Path("./domestic_pm10_improved")
OUT_PM25.mkdir(exist_ok=True)
OUT_PM10.mkdir(exist_ok=True)

# 로깅 설정
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('regression_model.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [16]:
# ───── 1. 데이터 로드 및 전처리 함수 ─────
def load_data():
    """데이터 파일 로드 및 기본 전처리"""
    logger.info("데이터 로드 시작")
    
    try:
        pm_kr_raw = pd.read_csv(DATA/"PM_KR_2015_2022.csv")
        pm_cn_raw = pd.read_csv(DATA/"PM_CN_2015_2022.csv")
        met_kr = pd.read_csv(DATA/"KOR_met_2015_2022.csv", parse_dates=["date"])
        master = pd.read_csv(DATA/"df_master.csv")
        
        # 데이터 기본 정보 로깅
        logger.info(f"PM_KR 데이터: {pm_kr_raw.shape}, PM_CN 데이터: {pm_cn_raw.shape}")
        logger.info(f"기상 데이터: {met_kr.shape}, 마스터 데이터: {master.shape}")
        
        return pm_kr_raw, pm_cn_raw, met_kr, master
    
    except Exception as e:
        logger.error(f"데이터 로드 중 오류 발생: {e}")
        raise

def tidy_pm(df: pd.DataFrame) -> pd.DataFrame:
    """딕셔너리 문자열로 된 PM 값을 long-format & 숫자형으로 변환"""
    logger.info(f"PM 데이터 정리 시작: {df.shape}")
    
    melted = df.melt(id_vars="date", var_name="city", value_name="raw")
    melted["date"] = pd.to_datetime(melted["date"])

    def extract(s: str, key: str):
        if not isinstance(s, str):
            return np.nan
        try:
            val = ast.literal_eval(s.replace("'", '"')).get(key, "").strip()
            return pd.to_numeric(val, errors="coerce")
        except Exception:
            return np.nan

    melted["pm25"] = melted["raw"].map(lambda x: extract(x, "pm25"))
    melted["pm10"] = melted["raw"].map(lambda x: extract(x, "pm10"))
    
    # 결측치 비율 확인 및 로깅
    pm25_missing = melted["pm25"].isna().mean() * 100
    pm10_missing = melted["pm10"].isna().mean() * 100
    logger.info(f"PM2.5 결측치 비율: {pm25_missing:.2f}%, PM10 결측치 비율: {pm10_missing:.2f}%")
    
    return melted.drop(columns="raw")

def interpolate_city(df: pd.DataFrame) -> pd.DataFrame:
    """시계열 보간 (city 단위)"""
    logger.info("시계열 보간 시작")
    
    out = []
    for c, g in df.groupby("city"):
        g = g.sort_values("date").set_index("date")
        
        # 보간 전 결측치 수 확인
        missing_before = g[["pm25", "pm10"]].isna().sum()
        
        # 시계열 보간 적용
        g[["pm25", "pm10"]] = g[["pm25", "pm10"]].interpolate(method="time", limit=3)
        
        # 보간 후 결측치 수 확인
        missing_after = g[["pm25", "pm10"]].isna().sum()
        
        # 결과 로깅
        logger.debug(f"도시 {c} - 보간 전 결측치: {missing_before.to_dict()}, 보간 후: {missing_after.to_dict()}")
        
        g["city"] = c
        out.append(g.reset_index())
    
    result = pd.concat(out, ignore_index=True)
    logger.info(f"시계열 보간 완료: {result.shape}")
    return result

def prepare_china_features(pm_cn: pd.DataFrame) -> pd.DataFrame:
    """중국 도시별 PM 데이터에서 시차(lag) 특성 생성"""
    logger.info("중국 시차 특성 생성 시작")
    
    # 피벗 테이블 생성
    pm_cn_pivot = pm_cn.pivot_table(index="date", columns="city", values=["pm25", "pm10"])
    pm_cn_pivot.columns = [f"{city}_{var}" for var, city in pm_cn_pivot.columns]
    
    # 기본 데이터프레임 생성
    pm_cn_features = pm_cn_pivot.copy()
    
    # 시차(lag) 특성 추가 (1일, 2일, 3일)
    for lag in [1, 2, 3]:
        lagged = pm_cn_pivot.shift(lag)
        lagged.columns = [f"{col}_lag{lag}" for col in pm_cn_pivot.columns]
        pm_cn_features = pd.concat([pm_cn_features, lagged], axis=1)
    
    # 이동 평균 특성 추가 (3일, 7일)
    for window in [3, 7]:
        rolled = pm_cn_pivot.rolling(window=window).mean()
        rolled.columns = [f"{col}_ma{window}" for col in pm_cn_pivot.columns]
        pm_cn_features = pd.concat([pm_cn_features, rolled], axis=1)
    
    logger.info(f"중국 시차 특성 생성 완료: {pm_cn_features.shape}")
    return pm_cn_features.reset_index()

def process_master_data(master: pd.DataFrame) -> tuple:
    """마스터 데이터 처리 및 일별 데이터로 보간"""
    logger.info("마스터 데이터 처리 시작")
    
    # PM10과 PM2.5 데이터 분리
    master_pm10 = master[master['pm_type'] == 'pm10'].copy()
    master_pm25 = master[master['pm_type'] == 'pm2.5'].copy()
    
    # 날짜 변환
    master_pm10["date"] = pd.to_datetime(master_pm10["year"].astype(str) + "-01-01")
    master_pm25["date"] = pd.to_datetime(master_pm25["year"].astype(str) + "-01-01")
    
    # 불필요한 열 제거
    drop_cols = ["year", "총합", "농도", "pm_type"]
    master_pm10 = master_pm10.drop(columns=drop_cols)
    master_pm25 = master_pm25.drop(columns=drop_cols)
    
    # 일별 데이터로 보간
    master_daily_pm10 = interpolate_to_daily(master_pm10)
    master_daily_pm25 = interpolate_to_daily(master_pm25)
    
    logger.info(f"마스터 데이터 처리 완료: PM10={master_daily_pm10.shape}, PM2.5={master_daily_pm25.shape}")
    return master_daily_pm10, master_daily_pm25

def interpolate_to_daily(df):
    """연간 데이터를 일별 데이터로 보간"""
    agg = (df.groupby(["시도", "date"]).mean(numeric_only=True)
            .reset_index()
            .rename(columns={"시도": "city"}))
    
    daily_blocks = []
    for c, g in agg.groupby("city"):
        g = g.set_index("date")
        
        # 다음 연도 1월 1일까지 일자 생성
        full_idx = pd.date_range(g.index.min(),
                    g.index.max() + pd.offsets.YearEnd(0),
                    freq="D")
        g = g.reindex(full_idx)
        
        # 선형 보간
        g = g.interpolate(method="linear")
        
        g["city"] = c
        daily_blocks.append(g.reset_index().rename(columns={"index": "date"}))
    
    return pd.concat(daily_blocks, ignore_index=True)

def merge_datasets(pm_kr, met_kr, master_daily, pm_cn_features, target_type):
    """데이터셋 병합"""
    logger.info(f"{target_type} 데이터셋 병합 시작")
    
    merged_df = (pm_kr
              .merge(met_kr, on=["city", "date"], how="left")
              .merge(master_daily, on=["city", "date"], how="left")
              .merge(pm_cn_features, on="date", how="left")
             )
    
    # 결측치 비율 확인
    missing_ratio = merged_df.isna().mean().sort_values(ascending=False)
    high_missing = missing_ratio[missing_ratio > 0.3]
    
    if not high_missing.empty:
        logger.warning(f"높은 결측치 비율(>30%)을 가진 특성: {high_missing.index.tolist()}")
    
    logger.info(f"{target_type} 데이터셋 병합 완료: {merged_df.shape}")
    return merged_df


In [11]:
# ───── 2. 특성 엔지니어링 함수 ─────
def engineer_features(df):
    """추가 특성 엔지니어링"""
    logger.info("특성 엔지니어링 시작")
    
    # 날짜 관련 특성
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    df['season'] = df['month'].map(lambda m: 1 if 3 <= m <= 5 else 2 if 6 <= m <= 8 else 3 if 9 <= m <= 11 else 4)
    
    # 기상 관련 파생 특성
    if 'rh' in df.columns and 'ta' in df.columns:
        # 불쾌지수 계산 (Discomfort Index)
        df['discomfort_idx'] = 0.81 * df['ta'] + 0.01 * df['rh'] * (0.99 * df['ta'] - 14.3) + 46.3
        
        # 체감온도 계산 (간단한 버전)
        df['feels_like'] = df['ta'] - 0.2 * (100 - df['rh']) / 5
    
    # 풍속 구간화
    if 'ws' in df.columns:
        df['wind_category'] = pd.cut(df['ws'], bins=[0, 0.5, 2, 3.5, 5, 100], 
                                    labels=['calm', 'light', 'moderate', 'strong', 'very_strong'])
        
        # 원-핫 인코딩
        wind_dummies = pd.get_dummies(df['wind_category'], prefix='wind')
        df = pd.concat([df, wind_dummies], axis=1)
    
    # 도시 원-핫 인코딩
    city_dummies = pd.get_dummies(df['city'], prefix='city')
    df = pd.concat([df, city_dummies], axis=1)
    
    # 상호작용 특성
    if 'rh' in df.columns and 'ws' in df.columns:
        df['rh_ws_interaction'] = df['rh'] * df['ws']
    
    logger.info(f"특성 엔지니어링 완료: {df.shape}")
    return df

def select_features(df, target):
    """특성 선택 및 타겟 준비"""
    logger.info(f"{target} 특성 선택 시작")
    
    # 제외할 열
    exclude = ["pm25", "pm10", "date", "wind_category"]
    
    # 타겟에 따라 추가 제외 열 설정
    if target == "pm10":
        exclude += [c for c in df.columns if "_pm25" in c]
    elif target == "pm25":
        exclude += [c for c in df.columns if "_pm10" in c]
    
    # 범주형 특성을 범주형으로 변환
    df["city"] = df["city"].astype("category")
    if "season" in df.columns:
        df["season"] = df["season"].astype("category")
    
    # 특성 선택
    features = [c for c in df.columns if c not in exclude]
    
    X = df[features]
    y = df[target]
    
    logger.info(f"{target} 특성 선택 완료: X={X.shape}, y={y.shape}")
    logger.info(f"{target} 결측치 없는 타겟 행: {y.notna().sum():,}")
    
    return X, y, features


In [12]:
def optimize_hyperparameters(X, y, cv):
    """하이퍼파라미터 최적화"""
    logger.info("하이퍼파라미터 최적화 시작")
    
    # 결측치 처리를 위한 간단한 전처리 파이프라인
    X_sample = X.sample(min(10000, len(X)))  # 최적화를 위한 샘플링
    y_sample = y.loc[X_sample.index]
    
    # 유효한 행만 선택
    valid_idx = y_sample.notna()
    X_sample = X_sample[valid_idx]
    y_sample = y_sample[valid_idx]
    
    # 하이퍼파라미터 탐색 공간
    param_distributions = {
        'num_leaves': [31, 63, 127],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 500],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_alpha': [0, 0.1, 0.5],
        'reg_lambda': [0, 0.1, 0.5],
        'min_child_samples': [5, 10, 20]
    }
    
    # 랜덤 탐색 교차 검증
    model = lgb.LGBMRegressor(objective='regression', random_state=42)
    
    search = RandomizedSearchCV(
        model, 
        param_distributions=param_distributions,
        n_iter=10,  # 10회 시도
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        verbose=1,
        random_state=42
    )
    
    search.fit(X_sample, y_sample)
    
    logger.info(f"최적 하이퍼파라미터: {search.best_params_}")
    logger.info(f"최적 RMSE: {-search.best_score_:.4f}")
    
    return search.best_params_

def train_evaluate_model(X, y, best_params, output_dir, target, cv_splits=5):
    """모델 학습 및 평가"""
    logger.info(f"{target} 모델 학습 및 평가 시작")
    
    # 시계열 교차 검증 설정
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    
    # 교차 검증으로 best_iteration 추정
    best_iters = []
    cv_scores = []
    
    for fold, (tr, va) in enumerate(tscv.split(X)):
        logger.info(f"교차 검증 {fold+1}/{cv_splits} 시작")
        
        # 유효한 행만 선택 (결측치 제외)
        valid_tr = y.iloc[tr].notna()
        valid_va = y.iloc[va].notna()
        
        X_tr = X.iloc[tr[valid_tr]]
        y_tr = y.iloc[tr[valid_tr]]
        X_va = X.iloc[va[valid_va]]
        y_va = y.iloc[va[valid_va]]
        
        # 모델 설정
        model = lgb.LGBMRegressor(
            objective="regression",
            **best_params,
            n_estimators=2000,  # 넉넉히 설정 (early stopping 사용)
            random_state=42
        )
        
        # 학습
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="rmse",
            callbacks=[lgb.early_stopping(50, verbose=False)],
        )
        
        # 최적 반복 횟수 저장
        best_iters.append(model.best_iteration_)
        
        # 검증 세트 평가
        y_pred = model.predict(X_va)
        rmse = np.sqrt(mean_squared_error(y_va, y_pred))
        cv_scores.append(rmse)
        
        logger.info(f"교차 검증 {fold+1} RMSE: {rmse:.4f}, 최적 반복 횟수: {model.best_iteration_}")
    
    # 최종 best_iteration 결정
    best_iter = int(np.median(best_iters))
    mean_cv_rmse = np.mean(cv_scores)
    
    logger.info(f"교차 검증 평균 RMSE: {mean_cv_rmse:.4f}")
    logger.info(f"최종 선택된 반복 횟수: {best_iter}")
    
    # 전체 데이터로 최종 모델 학습
    final_model = lgb.LGBMRegressor(
        objective="regression",
        **best_params,
        n_estimators=best_iter,
        random_state=42
    )
    
    # 유효한 행만 선택
    valid_idx = y.notna()
    X_valid = X[valid_idx]
    y_valid = y[valid_idx]
    
    final_model.fit(X_valid, y_valid)
    
    # 모델 저장
    model_path = output_dir / f"lgbm_model_{target}.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
    
    # 텍스트 형식으로도 저장
    model_txt = final_model.booster_.model_to_string(num_iteration=-1)
    with open(output_dir / f"lgbm_model_{target}.txt", "w", encoding="utf-8") as f:
        f.write(model_txt)
    
    logger.info(f"모델 저장 완료: {model_path}")
    
    # 예측 및 평가
    y_pred = final_model.predict(X)
    
    # 타깃·예측 모두 유효한(=NaN 아님) 행만 남김
    valid = (~np.isnan(y_pred)) & (y.notna())
    y_true = y[valid]
    y_pred_valid = y_pred[valid]
    
    # 지표 계산
    rmse = np.sqrt(mean_squared_error(y_true, y_pred_valid))
    mae = mean_absolute_error(y_true, y_pred_valid)
    r2 = r2_score(y_true, y_pred_valid)
    
    metrics = {
        "RMSE": float(rmse), 
        "MAE": float(mae), 
        "R2": float(r2),
        "CV_RMSE": float(mean_cv_rmse)
    }
    
    with open(output_dir/f"metrics_{target}.json", "w") as f:
        json.dump(metrics, f, indent=2)
    
    logger.info(f"{target} 최종 평가 지표: {metrics}")
    
    return final_model, metrics, X_valid, y_true, y_pred_valid

In [13]:
# ───── 4. 모델 해석 및 시각화 함수 ─────
def analyze_model(model, X, y_true, y_pred, features, output_dir, target):
    """모델 해석 및 시각화"""
    logger.info(f"{target} 모델 해석 시작")
    
    # SHAP 분석
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    # SHAP 요약 플롯
    plt.figure(figsize=(14, 10))
    shap.summary_plot(shap_values, X, show=False, max_display=20)
    plt.title(f'SHAP Summary for {target.upper()}', fontsize=16)
    plt.tight_layout()
    plt.savefig(output_dir/f"shap_summary_{target}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # SHAP 의존성 플롯 (상위 5개 특성)
    feature_importance = np.abs(shap_values).mean(0)
    feature_importance_df = pd.DataFrame({
        'feature': features,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    top_features = feature_importance_df['feature'].head(5).tolist()
    
    for feature in top_features:
        plt.figure(figsize=(10, 7))
        shap.dependence_plot(feature, shap_values, X, show=False)
        plt.title(f'SHAP Dependence Plot: {feature} for {target.upper()}', fontsize=14)
        plt.tight_layout()
        plt.savefig(output_dir/f"shap_dependence_{feature}_{target}.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # 특성 중요도 시각화
    plt.figure(figsize=(14, 10))
    lgb.plot_importance(model, max_num_features=20, figsize=(14, 10))
    plt.title(f'{target.upper()} LightGBM Feature Importance', fontsize=16)
    plt.tight_layout()
    plt.savefig(output_dir/f"feature_importance_{target}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 예측 vs 실제 산점도
    plt.figure(figsize=(10, 8))
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    plt.xlabel(f'Actual {target.upper()}', fontsize=12)
    plt.ylabel(f'Predicted {target.upper()}', fontsize=12)
    plt.title(f'Actual vs Predicted {target.upper()} (R² = {r2_score(y_true, y_pred):.4f})', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir/f"actual_vs_predicted_{target}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 잔차 분석
    residuals = y_true - y_pred
    
    plt.figure(figsize=(10, 8))
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel(f'Predicted {target.upper()}', fontsize=12)
    plt.ylabel('Residuals', fontsize=12)
    plt.title(f'Residual Plot for {target.upper()}', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir/f"residual_plot_{target}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 잔차 분포
    plt.figure(figsize=(10, 8))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuals', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title(f'Residual Distribution for {target.upper()}', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir/f"residual_distribution_{target}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    logger.info(f"{target} 모델 해석 완료")


In [14]:
def analyze_seasonal_performance(df, model, features, output_dir, target):
    """계절별 모델 성능 분석"""
    logger.info(f"{target} 계절별 성능 분석 시작")
    
    # 계절 정의
    seasons = {
        'spring': df[(df['date'].dt.month >= 3) & (df['date'].dt.month <= 5)],
        'summer': df[(df['date'].dt.month >= 6) & (df['date'].dt.month <= 8)],
        'fall': df[(df['date'].dt.month >= 9) & (df['date'].dt.month <= 11)],
        'winter': df[(df['date'].dt.month == 12) | (df['date'].dt.month <= 2)]
    }
    
    season_metrics = {}
    
    for season_name, season_df in seasons.items():
        if len(season_df) > 0:
            logger.info(f"{season_name} 분석 시작: {len(season_df)} 행")
            
            X_season = season_df[features]
            y_season = season_df[target]
            
            # 유효한 행만 선택
            valid_idx = y_season.notna()
            X_valid = X_season[valid_idx]
            y_valid = y_season[valid_idx]
            
            if len(y_valid) > 0:
                # 모델 예측
                y_pred = model.predict(X_valid)
                
                # 메트릭 계산
                rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                mae = mean_absolute_error(y_valid, y_pred)
                r2 = r2_score(y_valid, y_pred)
                
                season_metrics[season_name] = {"RMSE": float(rmse), "MAE": float(mae), "R2": float(r2)}
                
                with open(output_dir/f"{season_name}_{target}_metrics.json", "w") as f:
                    json.dump(season_metrics[season_name], f, indent=2)
                
                logger.info(f"{season_name} 성능: RMSE={rmse:.4f}, R2={r2:.4f}")
                
                # 계절별 SHAP 값
                explainer = shap.TreeExplainer(model)
                season_shap_values = explainer.shap_values(X_valid)
                
                plt.figure(figsize=(14, 10))
                shap.summary_plot(season_shap_values, X_valid, show=False, max_display=15)
                plt.title(f'SHAP Summary for {season_name.capitalize()} - {target.upper()}', fontsize=16)
                plt.tight_layout()
                plt.savefig(output_dir/f"{season_name}_{target}_shap_summary.png", dpi=300, bbox_inches='tight')
                plt.close()
    
    # 계절별 성능 비교 시각화
    if season_metrics:
        seasons_order = ['spring', 'summer', 'fall', 'winter']
        available_seasons = [s for s in seasons_order if s in season_metrics]
        
        metrics_to_plot = ['RMSE', 'R2']
        
        for metric in metrics_to_plot:
            plt.figure(figsize=(12, 8))
            values = [season_metrics[s][metric] for s in available_seasons]
            
            if metric == 'RMSE':
                title = f'계절별 RMSE 비교 ({target.upper()})'
                ylabel = 'RMSE (낮을수록 좋음)'
                color = 'skyblue'
            else:
                title = f'계절별 R² 비교 ({target.upper()})'
                ylabel = 'R² Score (높을수록 좋음)'
                color = 'lightgreen'
            
            bars = plt.bar(available_seasons, values, color=color, alpha=0.7)
            
            # 값 표시
            for bar, val in zip(bars, values):
                plt.text(bar.get_x() + bar.get_width()/2, val + 0.01, 
                        f'{val:.3f}', ha='center', fontsize=11)
            
            plt.title(title, fontsize=16)
            plt.ylabel(ylabel, fontsize=12)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.savefig(output_dir/f"seasonal_{metric.lower()}_{target}.png", dpi=300, bbox_inches='tight')
            plt.close()
    
    logger.info(f"{target} 계절별 성능 분석 완료")

def analyze_domestic_foreign_impact(df, features, target, output_dir, cv_splits=5):
    """국내 오염원과 국외 오염원의 영향 분석"""
    logger.info(f"{target} 국내외 오염원 영향 분석 시작")
    
    # 특성 분류
    domestic_cols = [col for col in features 
                    if not (col.endswith('_pm10') or col.endswith('_pm25') or 
                           col.endswith('_pm10_lag1') or col.endswith('_pm25_lag1') or
                           col.endswith('_pm10_lag2') or col.endswith('_pm25_lag2') or
                           col.endswith('_pm10_lag3') or col.endswith('_pm25_lag3') or
                           col.endswith('_pm10_ma3') or col.endswith('_pm25_ma3') or
                           col.endswith('_pm10_ma7') or col.endswith('_pm25_ma7'))]
    
    foreign_cols = [col for col in features 
                   if (col.endswith('_pm10') or col.endswith('_pm25') or 
                       col.endswith('_pm10_lag1') or col.endswith('_pm25_lag1') or
                       col.endswith('_pm10_lag2') or col.endswith('_pm25_lag2') or
                       col.endswith('_pm10_lag3') or col.endswith('_pm25_lag3') or
                       col.endswith('_pm10_ma3') or col.endswith('_pm25_ma3') or
                       col.endswith('_pm10_ma7') or col.endswith('_pm25_ma7'))]
    
    logger.info(f"국내 특성 수: {len(domestic_cols)}, 국외 특성 수: {len(foreign_cols)}")
    
    # 시계열 교차 검증 설정
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    
    # 모델 설정 (간소화된 파라미터)
    base_params = {
        'objective': 'regression',
        'num_leaves': 63, 
        'learning_rate': 0.05, 
        'n_estimators': 500, 
        'subsample': 0.8, 
        'colsample_bytree': 0.8,
        'random_state': 42
    }
    
    # 결과 저장용 딕셔너리
    results = {}
    
    # 1. 전체 특성 모델
    if len(features) > 0:
        X_all = df[features]
        y = df[target]
        
        # 유효한 행만 선택
        valid_idx = y.notna()
        X_all_valid = X_all[valid_idx]
        y_valid = y[valid_idx]
        
        model_all = lgb.LGBMRegressor(**base_params)
        
        cv_scores_all = []
        for tr, va in tscv.split(X_all_valid):
            model_all.fit(
                X_all_valid.iloc[tr], y_valid.iloc[tr],
                eval_set=[(X_all_valid.iloc[va], y_valid.iloc[va])],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(30, verbose=False)]
            )
            
            y_pred = model_all.predict(X_all_valid.iloc[va])
            rmse = np.sqrt(mean_squared_error(y_valid.iloc[va], y_pred))
            cv_scores_all.append(rmse)
        
        # 전체 데이터로 최종 모델 학습
        model_all.fit(X_all_valid, y_valid)
        y_pred_all = model_all.predict(X_all_valid)
        
        rmse_all = np.sqrt(mean_squared_error(y_valid, y_pred_all))
        r2_all = r2_score(y_valid, y_pred_all)
        
        results['전체 특성'] = {
            'RMSE': float(rmse_all),
            'R2': float(r2_all),
            'CV_RMSE': float(np.mean(cv_scores_all))
        }
        
        logger.info(f"전체 특성 모델 성능: RMSE={rmse_all:.4f}, R2={r2_all:.4f}")
    
    # 2. 국내 오염원만 사용한 모델
    if len(domestic_cols) > 0:
        X_domestic = df[domestic_cols]
        
        # 유효한 행만 선택
        X_domestic_valid = X_domestic[valid_idx]
        
        model_domestic = lgb.LGBMRegressor(**base_params)
        
        cv_scores_domestic = []
        for tr, va in tscv.split(X_domestic_valid):
            model_domestic.fit(
                X_domestic_valid.iloc[tr], y_valid.iloc[tr],
                eval_set=[(X_domestic_valid.iloc[va], y_valid.iloc[va])],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(30, verbose=False)]
            )
            
            y_pred = model_domestic.predict(X_domestic_valid.iloc[va])
            rmse = np.sqrt(mean_squared_error(y_valid.iloc[va], y_pred))
            cv_scores_domestic.append(rmse)
        
        # 전체 데이터로 최종 모델 학습
        model_domestic.fit(X_domestic_valid, y_valid)
        y_pred_domestic = model_domestic.predict(X_domestic_valid)
        
        rmse_domestic = np.sqrt(mean_squared_error(y_valid, y_pred_domestic))
        r2_domestic = r2_score(y_valid, y_pred_domestic)
        
        results['국내 오염원만'] = {
            'RMSE': float(rmse_domestic),
            'R2': float(r2_domestic),
            'CV_RMSE': float(np.mean(cv_scores_domestic))
        }
        
        logger.info(f"국내 오염원만 모델 성능: RMSE={rmse_domestic:.4f}, R2={r2_domestic:.4f}")
        
        # 모델 저장
        with open(output_dir/f"domestic_only_{target}_metrics.json", "w") as f:
            json.dump(results['국내 오염원만'], f, indent=2)
    
    # 3. 국외 오염원만 사용한 모델
    if len(foreign_cols) > 0:
        X_foreign = df[foreign_cols]
        
        # 유효한 행만 선택
        X_foreign_valid = X_foreign[valid_idx]
        
        model_foreign = lgb.LGBMRegressor(**base_params)
        
        cv_scores_foreign = []
        for tr, va in tscv.split(X_foreign_valid):
            model_foreign.fit(
                X_foreign_valid.iloc[tr], y_valid.iloc[tr],
                eval_set=[(X_foreign_valid.iloc[va], y_valid.iloc[va])],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(30, verbose=False)]
            )
            
            y_pred = model_foreign.predict(X_foreign_valid.iloc[va])
            rmse = np.sqrt(mean_squared_error(y_valid.iloc[va], y_pred))
            cv_scores_foreign.append(rmse)
        
        # 전체 데이터로 최종 모델 학습
        model_foreign.fit(X_foreign_valid, y_valid)
        y_pred_foreign = model_foreign.predict(X_foreign_valid)
        
        rmse_foreign = np.sqrt(mean_squared_error(y_valid, y_pred_foreign))
        r2_foreign = r2_score(y_valid, y_pred_foreign)
        
        results['국외 오염원만'] = {
            'RMSE': float(rmse_foreign),
            'R2': float(r2_foreign),
            'CV_RMSE': float(np.mean(cv_scores_foreign))
        }
        
        logger.info(f"국외 오염원만 모델 성능: RMSE={rmse_foreign:.4f}, R2={r2_foreign:.4f}")
        
        # 모델 저장
        with open(output_dir/f"foreign_only_{target}_metrics.json", "w") as f:
            json.dump(results['국외 오염원만'], f, indent=2)
    
    # 국내외 오염원 영향 비교 시각화
    if len(results) > 1:
        # R2 비교
        plt.figure(figsize=(12, 8))
        r2_values = [results[k]['R2'] for k in results.keys()]
        
        bars = plt.bar(list(results.keys()), r2_values, color=['#3498db', '#2ecc71', '#e74c3c'][:len(results)])
        
        # 값 표시
        for bar, val in zip(bars, r2_values):
            plt.text(bar.get_x() + bar.get_width()/2, val + 0.01, 
                    f'{val:.3f}', ha='center', fontsize=11)
        
        plt.title(f'{target.upper()} 국내외 오염원 영향 비교 (R²)', fontsize=16)
        plt.ylabel('R² Score (높을수록 좋음)', fontsize=12)
        plt.ylim(0, max(r2_values) * 1.2)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(output_dir/f"domestic_foreign_comparison_r2_{target}.png", dpi=300, bbox_inches='tight')
        plt.close()
        
        # RMSE 비교
        plt.figure(figsize=(12, 8))
        rmse_values = [results[k]['RMSE'] for k in results.keys()]
        
        bars = plt.bar(list(results.keys()), rmse_values, color=['#3498db', '#2ecc71', '#e74c3c'][:len(results)])
        
        # 값 표시
        for bar, val in zip(bars, rmse_values):
            plt.text(bar.get_x() + bar.get_width()/2, val + 0.5, 
                    f'{val:.2f}', ha='center', fontsize=11)
        
        plt.title(f'{target.upper()} 국내외 오염원 영향 비교 (RMSE)', fontsize=16)
        plt.ylabel('RMSE (낮을수록 좋음)', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(output_dir/f"domestic_foreign_comparison_rmse_{target}.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    logger.info(f"{target} 국내외 오염원 영향 분석 완료")
    return results

In [ ]:
def run_regression_analysis(target, output_dir):
    """회귀분석 실행 함수"""
    logger.info(f"\n===== {target.upper()} 회귀분석 시작 =====")
    
    # 1. 데이터 로드
    pm_kr_raw, pm_cn_raw, met_kr, master = load_data()
    
    # 2. 데이터 전처리
    pm_kr = tidy_pm(pm_kr_raw)
    pm_cn = tidy_pm(pm_cn_raw)
    
    # 3. 시계열 보간
    pm_kr = interpolate_city(pm_kr)
    pm_cn = interpolate_city(pm_cn)
    
    # 4. 중국 시차 특성 생성
    pm_cn_features = prepare_china_features(pm_cn)
    
    # 5. 마스터 데이터 처리
    master_daily_pm10, master_daily_pm25 = process_master_data(master)
    
    # 6. 데이터셋 병합
    if target == "pm10":
        df = merge_datasets(pm_kr, met_kr, master_daily_pm10, pm_cn_features, "PM10")
    else:
        df = merge_datasets(pm_kr, met_kr, master_daily_pm25, pm_cn_features, "PM2.5")
    
    # 7. 특성 엔지니어링
    df = engineer_features(df)
    
    # 8. 특성 선택
    X, y, features = select_features(df, target)
    
    # 9. 시계열 교차 검증 설정
    cv = TimeSeriesSplit(n_splits=5)
    
    # 10. 하이퍼파라미터 최적화
    best_params = optimize_hyperparameters(X, y, cv)
    
    # 11. 모델 학습 및 평가
    model, metrics, X_valid, y_true, y_pred = train_evaluate_model(
        X, y, best_params, output_dir, target, cv_splits=5
    )
    
    # 12. 모델 해석 및 시각화
    analyze_model(model, X_valid, y_true, y_pred, features, output_dir, target)
    
    # 13. 계절별 성능 분석
    analyze_seasonal_performance(df, model, features, output_dir, target)
    
    # 14. 국내외 오염원 영향 분석
    impact_results = analyze_domestic_foreign_impact(df, features, target, output_dir)
    
    # 15. 데이터셋 저장
    df.to_parquet(output_dir/f"final_dataset_{target}.parquet", index=False)
    
    logger.info(f"===== {target.upper()} 회귀분석 완료 =====\n")
    return model, metrics, impact_results

logger.info("회귀분석 시작")
    
# PM2.5 회귀분석
model_pm25, metrics_pm25, impact_pm25 = run_regression_analysis("pm25", OUT_PM25)
    
# PM10 회귀분석
model_pm10, metrics_pm10, impact_pm10 = run_regression_analysis("pm10", OUT_PM10)
    
# 결과 요약
logger.info("\n===== 회귀분석 결과 요약 =====")
logger.info(f"PM2.5 회귀분석 결과: {metrics_pm25}")
logger.info(f"PM10 회귀분석 결과: {metrics_pm10}")
logger.info("결과는 다음 폴더에 저장되었습니다:")
logger.info(f"PM2.5 결과: {OUT_PM25}")
logger.info(f"PM10 결과: {OUT_PM10}")
logger.info("===== 분석 완료 =====")
